# Let's load the Dataset

## Library

In [ ]:
# !pip install mne
# !pip install h5py

In [ ]:
import matplotlib
import pathlib 
#import mne
import numpy as np
import pandas as pd
# import os
# from os import listdir
# from os.path import isfile, join
# from tqdm import tqdm 
# import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Importing EEG and Dataset

In [ ]:
df=pd.read_excel('/content/drive/My Drive/deap_dataset.xlsx')

part1

In [ ]:
eeg=[]

raw_path = "/content/drive/My Drive/DEAP-Orginal/part1/" '''sample 1 to 23'''
for j in tqdm(os.listdir(raw_path)):
    raw = mne.io.read_raw(os.path.join(raw_path,j))
    raw=raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=['F3','F7','FC5','FC1','CP5','CP1','P3','P7','PO3','AF4','Fz','F4','FC6',
                                                                       'FC2','Cz','C4','CP6','CP2','P8',
                                                                       'EXG1','EXG2','EXG3', 'EXG4', 'EXG5', 'EXG6','EXG7','EXG8',
                                                                       'GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp','Status'])
    raw.set_montage('standard_1020')
    raw.resample(128)
    #raw.load_data()
    raw.filter(l_freq=3, h_freq=47)
    eeg.append(raw)


part2

In [ ]:
raw_path = "/content/drive/My Drive/DEAP-Orginal/part2/" '''sample 24 to 28'''
for j in tqdm(os.listdir(raw_path)):
    raw = mne.io.read_raw_edf(os.path.join(raw_path,j), preload=True, stim_channel="")
    mne.rename_channels(raw.info,mapping={"":"Status"})
    raw=raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=['F3','F7','FC5','FC1','CP5','CP1','P3','P7','PO3','AF4','Fz','F4','FC6',
                                                                       'FC2','Cz','C4','CP6','CP2','P8',
                                                                       'EXG1','EXG2','EXG3', 'EXG4', 'EXG5', 'EXG6','EXG7','EXG8',
                                                                       'GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp','Status'])
    raw.set_montage('standard_1020')
    raw.resample(128)
    #raw.load_data()
    raw.filter(l_freq=3, h_freq=47)
    eeg.append(raw)

part3

In [ ]:
raw_path = "/content/drive/My Drive/DEAP-Orginal/part3/"  '''sample 29 to 32'''
for j in tqdm(os.listdir(raw_path)):
    raw = mne.io.read_raw_edf(os.path.join(raw_path,j), preload=True, stim_channel="-1")
    mne.rename_channels(raw.info,mapping={"-1":"Status"})
    raw=raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=['F3','F7','FC5','FC1','CP5','CP1','P3','P7','PO3','AF4','Fz','F4','FC6',
                                                                       'FC2','Cz','C4','CP6','CP2','P8',
                                                                       'EXG1','EXG2','EXG3', 'EXG4', 'EXG5', 'EXG6','EXG7','EXG8',
                                                                       'GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp','-0','Status'])
    raw.set_montage('standard_1020')
    raw.resample(128)
    #raw.load_data()
    raw.filter(l_freq=3, h_freq=47)
    eeg.append(raw)

## Cropping EEG

FIRST

In [ ]:
X=[]
Y_1=[]
Y_2=[]
Y_3=[]
for j in range(len(eeg)):
  for i in range(40):
    n=df['start'][i+j*40]
    m=df['finish'][i+j*40]
    raw_cropped = eeg[j].copy().crop(tmin=n,tmax=m) #s
    X.append(raw_cropped.get_data())
    Y_1.append(df['label1'][i+j*40])
    Y_2.append(df['label2'][i+j*40])
    Y_3.append(df['label3'][i+j*40])

SECOND

In [ ]:
X=[]
y=[]
for k in range(1,33):
  for i in range(40*(k-1),40*k):
    n=df['start'][i]
    m=df['finish'][i]
    raw_cropped = eeg[k-1].copy().crop(tmin=n,tmax=m)
    X.append(raw_cropped.get_data())
    y.append(df['label1'][i])

## Save and Load X,Y

Convert to Array

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y_1)

In [ ]:
Y2=np.asarray(Y_2)
Y3=np.asarray(Y_3)

Save the Arrays

In [ ]:
np.savez("/content/drive/My Drive/DEAP-Orginal/xy/X_eeg",X)
np.savez("/content/drive/My Drive/DEAP-Orginal/xy/Y_1",Y)

In [ ]:
np.savez("/content/drive/My Drive/DEAP-Orginal/xy/Y_2",Y2)
np.savez("/content/drive/My Drive/DEAP-Orginal/xy/Y_3",Y3)

Load the Arrays

In [ ]:
#load X data
a = np.load("/content/drive/My Drive/DEAP-Orginal/xy/X_eeg.npz") #make sure you use the .npz!
X = a['arr_0']

In [ ]:
#load Y data '''Y_1 : 4 categories, Y_2: 10 categories, Y_3: 2 categories'''
b = np.load("/content/drive/My Drive/DEAP-Orginal/xy/Y_3.npz") #make sure you use the .npz!
Y = b['arr_0']

##One-hot label

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data=Y
values = array(data)
print(values)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

In [ ]:
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

# Building Model


## Library

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
import tensorflow as tf
from random import shuffle
from keras import models
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling1D,MaxPooling2D, ConvLSTM2D,Conv1D, AveragePooling2D,LSTM
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
#from keras.optimizers import Adam, SGD 
import time
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,UpSampling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping

## Create Models

### Model1

In [ ]:
model1 = keras.models.Sequential()

model1.add(Conv1D(64, kernel_size=3,padding='same', activation='relu',input_shape = (13, 7681)))
model1.add(BatchNormalization())
model1.add(MaxPooling1D(pool_size=2))

model1.add(Conv1D(32, kernel_size=3,padding='same', activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling1D(pool_size=2))

model1.add(Flatten())
# model1.add(Dense(64, activation="relu"))
model1.add(Dense(32, activation="relu"))
model1.add(Dense(2, activation = "softmax"))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 64)            1474816   
                                                                 
 batch_normalization (BatchN  (None, 13, 64)           256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 64)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 32)             6176      
                                                                 
 batch_normalization_1 (Batc  (None, 6, 32)            128       
 hNormalization)                                                 
                                                        

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
history = model1.fit(X_train, y_train_onehot, epochs=20, batch_size =128 , validation_data=(X_test,y_test_onehot))
#cweights_bykeras_dict
#model1_json = model1.to_json()
#with open("model1.json", "w") as json_file:
    #json_file.write(model1_json)

### Model2

In [ ]:
model2 = Sequential()
model2.add(Conv1D(128, kernel_size=7, padding='same', activation='relu', input_shape=(13, 7681)))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling1D(pool_size=2))

model2.add(Conv1D(64, kernel_size=5, padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling1D(pool_size=1))

model2.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling1D(pool_size=1))

model2.add(Flatten())

model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(2, activation='softmax'))
model2.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
history = model2.fit(X_train, y_train_onehot, epochs=900, batch_size =256 , validation_data=(X_test,y_test_onehot))


### Model3

In [ ]:
keras.regularizers.l1(0.01)
keras.regularizers.l2(0.01)
keras.regularizers.l1_l2(l1=0.01, l2=0.01)

In [ ]:
from keras.regularizers import l2,l1

In [ ]:
# make a sequential model
model3 = keras.models.Sequential()

# add 1-layer cnn
model3.add(Conv1D(128, kernel_size=8, padding='same',activation='relu' ,strides=1,input_shape=(13, 7681))) #(13, 7681)
# model3.add(Dropout(0.25))
model3.add(BatchNormalization())
model3.add(MaxPooling1D(pool_size=2))


# add 3-layer lstm
model3.add(LSTM(128, return_sequences=True, stateful=False))
model3.add(Dropout(0.25))
model3.add(BatchNormalization())

model3.add(LSTM(64, return_sequences=True, stateful=False))
model3.add(Dropout(0.25))
model3.add(BatchNormalization())

# model3.add(LSTM(32, return_sequences=True, stateful=False))
# model3.add(Dropout(0.25))
# model3.add(BatchNormalization())


model3.add(Flatten())
# model3.add(Dense(64, activation="relu")) #,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)
model3.add(Dense(2, activation='softmax'))
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 128)           7865472   
                                                                 
 batch_normalization (BatchN  (None, 13, 128)          512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 128)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 6, 128)            131584    
                                                                 
 dropout (Dropout)           (None, 6, 128)            0         
                                                                 
 batch_normalization_1 (Batc  (None, 6, 128)           5

In [ ]:
# make a sequential model
model3 = keras.models.Sequential()

# add 1-layer cnn
model3.add(Conv1D(128, kernel_size=7, padding='same',activation='relu' ,strides=1,input_shape=(7681,13))) #(13, 7681)
model3.add(Dropout(0.15))
model3.add(BatchNormalization())
model3.add(MaxPooling1D(pool_size=2))

# model3.add(Conv1D(128, kernel_size=5, padding='same',activation='relu' ,strides=1))
# model3.add(MaxPooling1D(pool_size=2))

# model3.add(Conv1D(64, kernel_size=3, padding='same',activation='relu' ,strides=1))
# model3.add(MaxPooling1D(pool_size=2))

# add 3-layer lstm
model3.add(LSTM(64, return_sequences=True, stateful=False))
model3.add(Dropout(0.25))
model3.add(BatchNormalization())

model3.add(LSTM(64, return_sequences=True, stateful=False))
model3.add(Dropout(0.25))
model3.add(BatchNormalization())

# model3.add(LSTM(32, return_sequences=True, stateful=False))
# model3.add(Dropout(0.25))
# model3.add(BatchNormalization())


model3.add(Flatten())
# model3.add(Dense(128, activation="relu"))
model3.add(Dense(32, activation="relu")) #,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)
model3.add(Dense(4, activation='softmax'))
model3.summary()

### Model4

In [ ]:
model4 = Sequential()

model4.add(LSTM(256, input_shape = (13, 7681),return_sequences=True))
model4.add(BatchNormalization())
model4.add(Dropout(0.3))


model4.add(LSTM(128,activation="relu",return_sequences=True))
model4.add(BatchNormalization())
model4.add(Dropout(0.3))

model4.add(LSTM(64,activation="relu",return_sequences=True))
model4.add(BatchNormalization())
model4.add(Dropout(0.3))


model4.add(LSTM(32,activation="relu"))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

model4.add(Dense(2, activation="softmax"))


model4.summary()

## Fit the model on data 

In [ ]:
#sample_size=[0.1,0.15,0.2,0.25,0.3,0.33,0.35,0.4,0.45,0.5]
lst_test=[]
lst_train=[]
each_train_acc=[]
each_train_loss=[]
loss_lst_train=[]
opt = keras.optimizers.Adam(learning_rate=0.001)
opt1 = keras.optimizers.Adam(learning_rate=0.002)
opt2 = keras.optimizers.RMSprop(learning_rate=0.001)
opt3 = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00, amsgrad=False)
opt4 = keras.optimizers.RMSprop(learning_rate=0.003)
cv=range(10)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
line = ['solid', (0, (5, 5)), (0, (3, 5, 1, 5)), (0, (3, 1, 1, 1)), (0, (1, 1)), (0, (5, 10)), (0, (5, 1)), (0, (3, 5, 1, 5, 1, 5)), (0, (3, 1, 1, 1, 1, 1)), (0, (3, 10, 1, 10, 1, 10)), (0, (1, 10)), 'dashdot']

In [ ]:
start=time.time()
for j in cv:
    
    X_train, X_test, y_train, y_test = train_test_split (X,onehot_encoded , test_size=0.33,shuffle=True)  #onehot_encoded
    # y_test_onehot = keras.utils.to_categorical(y_test,num_classes=5)
    # y_train_onehot = keras.utils.to_categorical(y_train,num_classes=5)
    model1.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])  # binary_crossentropy #categorical_crossentropy
    history = model1.fit(X_train, y_train, epochs=100, batch_size =256,validation_data=(X_test,y_test)) 
    #class_weight=cweights_bykeras_dict, callbacks = [EarlyStopping(monitor='val_acc', patience=2)]

    score_test = model1.evaluate(X_test,y_test, verbose=1)
    lst_test.append(score_test)
    
    
    score_train=(history.history['accuracy'])
    each_train_acc.append(score_train)
    mean_acc=np.mean(score_train)
    lst_train.append(mean_acc)

    loss_train=(history.history['loss'])
    each_train_loss.append(loss_train)
    mean_loss=np.mean(loss_train)
    loss_lst_train.append(mean_loss)

    plt.plot(history.history['accuracy'], color='black',linestyle=line[0],linewidth=3)
    plt.plot(history.history['val_accuracy'], color='gray',linestyle=line[4],linewidth=3)
    plt.ylim([0.1, 1.1])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Test', ''], loc='upper left')
    plt.show()


    # summarize history for loss
    plt.plot(history.history['loss'], color='black',linestyle=line[0],linewidth=3)
    plt.plot(history.history['val_loss'], color='gray',linestyle=line[3],linewidth=3)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Test',''], loc='upper left')
    plt.show()
    

stop=time.time()
print(stop-start)

In [ ]:
 score_test = model1.evaluate(X_test,y_test, verbose=2)

## Mean of Test set Accuracy

In [ ]:
test_mean_acc=[]
for i in range(0,len(lst_test)):
  test_mean_acc.append(lst_test[i][1])

test_mean_acc_model=np.mean([test_mean_acc])
print(test_mean_acc_model)

In [ ]:
train_mean_acc=[]
for i in range(0,len(lst_train)):
  train_mean_acc.append(lst_train[i])

train_mean_acc_model=np.mean([train_mean_acc])
print(train_mean_acc_model)

## Mean of Test set Loss function

In [ ]:
test_mean_loss=[]
for i in range(len(lst_test)):
  test_mean_loss.append(lst_test[i][0])

test_mean_loss_model=np.mean([test_mean_loss])
print(test_mean_loss_model)

In [ ]:
test_mean_loss

In [ ]:
train_mean_loss=[]
for i in range(len(loss_lst_train)):
  train_mean_loss.append(loss_lst_train[i])

train_mean_loss_model9=np.mean([train_mean_loss])
print(train_mean_loss_model9)

### Accuracy

In [ ]:
print(history.history.keys())
import matplotlib.pyplot as plt
import numpy
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'],color='green')
plt.ylim([0.4, 1.1])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='best',fontsize='medium')
plt.show()

# from matplotlib import pyplot
# plt.plot(history.history['accuracy'])
# plt.show()


### Loss

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'],color='green')
plt.ylim([-0.1,1])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'],loc='best',fontsize='medium')
plt.show()


Test plot

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['val_loss'])
plt.title('Validation')
plt.ylabel('Validation accuracy')
plt.xlabel('Validation loss')
plt.show()